## Orbit analysis of Aquarius II and Bootes II using Galpy

# Using values from https://ui.adsabs.harvard.edu/abs/2023arXiv230203708B/abstract, this notebook computes the orbit for Aquarius II, the process was repeated for Boo II

In [11]:
from galpy.orbit import Orbit
from galpy.util import conversion
import numpy as np
import matplotlib.pyplot as plt
from astropy.coordinates import SkyCoord
from astropy import units
from galpy.util.conversion import get_physical
from galpy.potential import MWPotential2014, ChandrasekharDynamicalFrictionForce, HernquistPotential, \
                            MovingObjectPotential, evaluateRforces, evaluatephitorques, evaluatezforces, \
                            NonInertialFrameForce
from galpy.potential.mwpotentials import McMillan17

In [2]:
# initializing the guassian distributions for the phase space parameters
tdisrupt=5 # Using 5 Gyr

# Using McMillan17 potential
pot = McMillan17

ro = conversion.get_physical(pot)['ro']
vo = conversion.get_physical(pot)['vo']
to = conversion.time_in_Gyr(ro=ro, vo=vo)
mo = conversion.mass_in_msol(ro=ro, vo=vo)
ts = np.linspace(0, -tdisrupt/to, 1001) # backwards to 5 Gyrs 

sixD = [338.4813, -9.3274, 107.9, -0.27, -0.44, -65.3] # the 6D parameters from the paper
uncer = [0.005, 0.005, 3.3, 0.12, 0.1, 1.8] # the uncertainties in these parameters

n_unc = 1000 # number of instances
sixD_unc = np.random.normal(loc=sixD, scale=uncer, size = (n_unc,6))

In [3]:
orbs = Orbit(sixD, radec=True, ro=ro, vo=vo)
orbs_unc = Orbit(sixD_unc, radec=True,  ro=ro, vo=vo)
orbs.integrate(ts, pot)
orbs_unc.integrate(ts, pot)

In [4]:
from scipy.signal import argrelextrema

# for local maxima
apo_indx = argrelextrema(orbs.r(ts), np.greater)[0]
print('Indices where a maximum occurs are at', apo_indx)

# for local minima
peri_indx = argrelextrema(orbs.r(ts), np.less)[0]
print('Indices where a minimum occurs are at', peri_indx)

#print(test.r(-ts)[peri_indx[0][1]]) #test.r(-ts)[peri_indx[1]], test.r(-ts)[659], test.r(-ts)[917])
print('First pericentre is at', orbs.r(ts)[peri_indx[0]], 'kpc')
print('First apocentre is at', orbs.r(ts)[apo_indx[0]], 'kpc')

Indices where a maximum occurs are at [396]
Indices where a minimum occurs are at [ 61 732]
First pericentre is at 98.65350566997277 kpc
First apocentre is at 154.57747787102295 kpc


In [5]:
from scipy.signal import argrelextrema

sat_peri = []
for i in range(len(orbs_unc)):
    o = orbs_unc.r(ts)[i,:] # Get the radial distance array for orbit i, assign it to 'o'
    m_list = o[(argrelextrema(o, np.less)[0])] # Find all local minima
    if m_list.tolist() == []: # If there are no local minima
        sat_peri.append(0)    # Set the pericentre to 0, you can change this
                              #  to min(o) if you want the pericentre
                              #  to be the minimum.
    else: # If there is a local minimum
        sat_peri.append(m_list[0]) # Set orbit i's local minimum to be the nearest one

In [7]:
sat_ap = []
for i in range(len(orbs_unc)):
    o = orbs_unc.r(ts)[i,:]
    m_list = o[(argrelextrema(o, np.greater)[0])]
    if m_list.tolist() == []:
        sat_ap.append(0)    #max(o) if you prefer
    else:
        sat_ap.append(m_list[0])

In [8]:
pap2 = []
pperi2 = []
eccs2 = []
for i in range(len(sat_ap)):
    if (not sat_ap[i] == 0) and (not sat_peri[i]==0): # if either the apocenter or pericenter is 0, it is not considered
        pap2.append(sat_ap[i])
        pperi2.append(sat_peri[i])
        eccs2.append(orbs_unc[i].e())

In [9]:
print('84%, 50%, 16% of peri:', np.percentile(pperi2, [84,50,16]))
print('84%, 50%, 16% of apo:', np.percentile(pap2, [84,50,16]))
print('84%, 50%, 16% of e:', np.percentile(eccs2, [84,50,16]))

84%, 50%, 16% of peri: [103.51092137  96.25694574  61.46863152]
84%, 50%, 16% of apo: [267.53203869 144.81923295 113.32510967]
84%, 50%, 16% of e: [0.49283955 0.29946476 0.18415884]


## Now with LMC

In [66]:
mass_lmc=1.38e11 #solar masses
rscale_lmc=10.2 #kpc

pot = McMillan17

#Initialize and integrate the orbit of the LMC
#Note orbit has to be integrated back 10 Gyr
#Note we assume the LMC experienced dynamical friction due to MW
o_lmc = Orbit.from_name('LMC', ro=ro, vo=vo, solarmotion=[-11.1, 24.0, 7.25])
ts= np.linspace(0.,-tdisrupt/to,1001)
cdf= ChandrasekharDynamicalFrictionForce(GMs=mass_lmc/mo, rhm=rscale_lmc/ro, dens=pot[1], ro=ro,vo=vo)
o_lmc.integrate(ts,pot+cdf)

#Setup a moving Hernquist potential to represent the LMC
lmcpot = HernquistPotential(amp=2*10.**11.*units.Msun, a=5.*units.kpc/(1.+np.sqrt(2.))) #rhm = (1+sqrt(2)) a
moving_lmcpot = MovingObjectPotential(o_lmc, pot=lmcpot)
#Add the moving Hernquest potential to the MW
total_pot = [pot]
total_pot += [moving_lmcpot]

In [71]:
from galpy.potential import (evaluateRforces, evaluatephitorques,
                                 evaluatezforces)
loc_origin= 1e-4 # Small offset in R to avoid numerical issues
ax= lambda t: evaluateRforces(moving_lmcpot,loc_origin,0.,phi=0.,t=t,
                                use_physical=False)
ay= lambda t: evaluatephitorques(moving_lmcpot,loc_origin,0.,phi=0.,t=t,
                                use_physical=False)/loc_origin
az= lambda t: evaluatezforces(moving_lmcpot,loc_origin,0.,phi=0.,t=t,
                                use_physical=False)

t_intunits= o_lmc.time(use_physical=False)[::-1] # need to reverse the order for interp
ax4int= np.array([ax(t) for t in t_intunits])
ax_int= lambda t: np.interp(t,t_intunits,ax4int)
ay4int= np.array([ay(t) for t in t_intunits])
ay_int= lambda t: np.interp(t,t_intunits,ay4int)
az4int= np.array([az(t) for t in t_intunits])
az_int= lambda t: np.interp(t,t_intunits,az4int)

In [72]:
nip = NonInertialFrameForce(a0=[ax_int, ay_int, az_int])

total_potential = McMillan17 + nip + moving_lmcpot

In [73]:
olmc = Orbit(sixD, radec=True, ro=ro, vo=vo)
olmc_unc = Orbit(sixD_unc, radec=True,  ro=ro, vo=vo)

# Integrating the orbits
olmc.integrate(ts, pot=total_potential)
olmc_unc.integrate(ts, pot=total_potential)

In [74]:
from scipy.signal import argrelextrema

sat_peri2 = []
for i in range(len(olmc_unc)):
    o = olmc_unc.r(ts)[i,:] # Get the radial distance array for orbit i, assign it to 'o'
    m_list = o[(argrelextrema(o, np.less)[0])] # Find all local minima
    if m_list.tolist() == []: # If there are no local minima
        sat_peri2.append(0)    # Set the pericentre to 0, you can change this
                              #  to min(o) if you want the pericentre
                              #  to be the minimum.
    else: # If there is a local minimum
        sat_peri2.append(m_list[0]) # Set orbit i's local minimum to be the nearest one
        
sat_ap2 = []
for i in range(len(olmc_unc)):
    o = olmc_unc.r(ts)[i,:]
    m_list = o[(argrelextrema(o, np.greater)[0])]
    if m_list.tolist() == []:
        sat_ap2.append(0)    #max(o))
    else:
        sat_ap2.append(m_list[0])
        
pap22 = []
pperi22 = []
eccs22 = []
for i in range(len(sat_ap2)):
    if (not sat_ap2[i] == 0) and (not sat_peri2[i]==0): # taking out any unbound instances
        pap22.append(sat_ap2[i])
        pperi22.append(sat_peri2[i])
        eccs22.append(olmc_unc[i].e())
        
print('84%, 50%, 16% of peri:', np.percentile(pperi22, [84,50,16]))
print('84%, 50%, 16% of apo:', np.percentile(pap22, [84,50,16]))
print('84%, 50%, 16% of e:', np.percentile(eccs22, [84,50,16]))

84%, 50%, 16% of peri: [99.47643447 75.12880737 35.28407976]
84%, 50%, 16% of apo: [274.1158475  175.88662547 144.73397411]
84%, 50%, 16% of e: [0.62397567 0.47600966 0.40738053]
